In [6]:
import pandas as pd
qbhalf = pd.read_csv("QBs_HalfPPR.csv")
rbhalf = pd.read_csv("RBs_HalfPPR.csv")
wrtehalf = pd.read_csv("WRTE_HalfPPR.csv")

qbnon = pd.read_csv("QBs_NonPPR.csv")
rbnon = pd.read_csv("RBs_NonPPR.csv")
wrtenon = pd.read_csv("WRTE_NonPPR.csv")

qbfull = pd.read_csv("QBs_PPR.csv")
rbfull = pd.read_csv("RBs_PPR.csv")
wrtefull = pd.read_csv("WRTE_PPR.csv")

halfrankings = pd.concat([qbhalf, rbhalf, wrtehalf], ignore_index=True)
nonrankings = pd.concat([qbnon, rbnon, wrtenon], ignore_index=True)
fullrankings = pd.concat([qbfull, rbfull, wrtefull], ignore_index=True)

halfrankings = halfrankings.sort_values(by='Final PPG', ascending=False)
nonrankings = nonrankings.sort_values(by='Final PPG', ascending=False)
fullrankings = fullrankings.sort_values(by='Final PPG', ascending=False)

halfrankings = halfrankings.reset_index(drop=True)
nonrankings = nonrankings.reset_index(drop=True)
fullrankings = fullrankings.reset_index(drop=True)

fullrankings

,Rank,Name,Team,Position,Final PPG
0,1,Josh Allen,buf,QB,20.652372
1,2,Dak Prescott,dal,QB,20.626648
2,3,Brock Purdy,sfo,QB,20.172126
3,4,Jordan Love,gnb,QB,20.123518
4,5,Lamar Jackson,rav,QB,20.034334
...,...,...,...,...,...
453,67,Blaine Gabbert,kan,QB,2.218068
454,68,Matt Barkley,jax,QB,2.078506
455,69,Malik Willis,oti,QB,1.749417
456,70,Cooper Rush,dal,QB,0.914377


In [7]:
corrections = {
    'Indianapolis Colts': 'clt',
    'New England Patriots': 'nwe',
    'Tampa Bay Buccaneers': 'tam',
    'New Orleans Saints': 'nor',
    'Los Angeles Rams': 'ram',
    'St. Louis Rams': 'ram',
    'Arizona Cardinals': 'crd',
    'Green Bay Packers': 'gnb',
    'Houston Texans': 'htx',
    'Las Vegas Raiders': 'rai',
    'Oakland Raiders': 'rai',
    'Baltimore Ravens': 'rav',
    'Los Angeles Chargers': 'sdg',
    'San Diego Chargers': 'sdg',
    'Kansas City Chiefs': 'kan',
    'Tennessee Titans': 'oti',
    'San Francisco 49ers': 'sfo',
    'Atlanta Falcons': 'atl',
    'Buffalo Bills': 'buf',
    'Carolina Panthers': 'car',
    'Chicago Bears': 'chi',
    'Cincinnati Bengals': 'cin',
    'Cleveland Browns': 'cle',
    'Dallas Cowboys': 'dal',
    'Denver Broncos': 'den',
    'Detroit Lions': 'det',
    'Jacksonville Jaguars': 'jax',
    'Miami Dolphins': 'mia',
    'Minnesota Vikings': 'min',
    'New York Giants': 'nyg',
    'New York Jets': 'nyj',
    'Philadelphia Eagles': 'phi',
    'Pittsburgh Steelers': 'pit',
    'Seattle Seahawks': 'sea',
    'Washington Commanders': 'was',
    'Washington Redskins': 'was',
    'Washington Football Team': 'was'
}

reverse_corrections = {v: k for k, v in corrections.items()}

# Map the team names in the DataFrame
fullrankings['Team'] = fullrankings['Team'].map(reverse_corrections)
halfrankings['Team'] = halfrankings['Team'].replace(reverse_corrections)
nonrankings['Team'] = nonrankings['Team'].replace(reverse_corrections)

bye_weeks = {
    'Detroit Lions': 5,
    'San Diego Chargers': 5,
    'Los Angeles Chargers': 5,
    'Philadelphia Eagles': 5,
    'Tennessee Titans': 5,
    'Kansas City Chiefs': 6,
    'St. Louis Rams': 6,
    'Los Angeles Rams': 6,
    'Miami Dolphins': 6,
    'Minnesota Vikings': 6,
    'Chicago Bears': 7,
    'Dallas Cowboys': 7,
    'Pittsburgh Steelers': 9,
    'San Francisco 49ers': 9,
    'Cleveland Browns': 10,
    'Green Bay Packers': 10,
    'Oakland Raiders': 10,
    'Las Vegas Raiders': 10,
    'Seattle Seahawks': 10,
    'Arizona Cardinals': 11,
    'Carolina Panthers': 11,
    'New York Giants': 11,
    'Tampa Bay Buccaneers': 11,
    'Atlanta Falcons': 12,
    'Buffalo Bills': 12,
    'Cincinnati Bengals': 12,
    'Jacksonville Jaguars': 12,
    'New Orleans Saints': 12,
    'New York Jets': 12,
    'Baltimore Ravens': 14,
    'Denver Broncos': 14,
    'Houston Texans': 14,
    'Indianapolis Colts': 14,
    'New England Patriots': 14,
    'Washington Football Team': 14,
    'Washington Commanders': 14
}

# Function to add bye weeks to DataFrame
def add_bye_weeks(df, bye_weeks):
    df['Bye Week'] = df['Team'].map(bye_weeks)
    return df

halfrankings = add_bye_weeks(halfrankings, bye_weeks)
nonrankings = add_bye_weeks(nonrankings, bye_weeks)
fullrankings = add_bye_weeks(fullrankings, bye_weeks)

fullrankings['Rank'] = range(1, len(fullrankings) + 1)
halfrankings['Rank'] = range(1, len(halfrankings) + 1)
nonrankings['Rank'] = range(1, len(nonrankings) + 1)



In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the webpage
url = 'https://www.footballguys.com/adp'

# Fetch the HTML content from the URL
response = requests.get(url)
if response.status_code == 200:
    html_content = response.text

    # Create a BeautifulSoup object
    soup = BeautifulSoup(html_content, 'html.parser')

    # List to hold player names and their ESPN values
    player_data = []
    rows = soup.find_all('tr')

    for row in rows:
        # Extract the player name
        name_td = row.find('td', class_='name sticky-col text-start')
        if name_td:
            name_a = name_td.find('a')
            if name_a:
                player_name = name_a.get_text().strip()
                # Extract the 9th <td> element (index 8) for ESPN value
                tds = row.find_all('td')
                if len(tds) >= 9:
                    espn_value = tds[8].get_text().strip()
                    # Append the player name and ESPN value as a tuple to the list
                    player_data.append((player_name, espn_value))

    # Create a DataFrame from the list
    ESPN = pd.DataFrame(player_data, columns=['Player Name', 'ESPN ADP'])

    # Convert ESPN Rank to numeric for proper sorting, handling non-numeric cases
    ESPN['ESPN ADP'] = pd.to_numeric(ESPN['ESPN ADP'], errors='coerce')
    ESPN = ESPN.dropna().sort_values('ESPN ADP')  # Drop rows where conversion failed and sort

    # Print the DataFrame
    print(ESPN)
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


             Player Name  ESPN ADP
0    Christian McCaffrey       1.0
2            Tyreek Hill       2.0
1            CeeDee Lamb       3.0
4            Breece Hall       4.0
6         Bijan Robinson       5.0
..                   ...       ...
359       Joe Milton III     246.0
245            Zach Ertz     247.0
445      Braxton Berrios     248.0
475          J.J. Taylor     249.0
371            Noah Gray     250.0

[250 rows x 2 columns]


In [9]:
fullrankings = fullrankings.merge(ESPN, left_on='Name', right_on='Player Name', how='left').drop(columns=['Player Name'])
halfrankings = halfrankings.merge(ESPN, left_on='Name', right_on='Player Name', how='left').drop(columns=['Player Name'])
nonrankings = nonrankings.merge(ESPN, left_on='Name', right_on='Player Name', how='left').drop(columns=['Player Name'])

In [10]:
fullrankings.to_csv("Full PPR Rankings.csv")
halfrankings.to_csv("Half PPR Rankings.csv")
nonrankings.to_csv("Non PPR Rankings.csv")